### **Task 1:**

Complete the following steps to establish a connection pool:

- To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

- To find the information on the error, import the Error class from MySQL Connector/Python.

- Define your database configurations as a Python dictionary object called dbconfig.

- Establish a connection pool [pool_name = pool_b] with two connections. 

- Implement error handling using a try-except block in case the connection fails. 

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    'database': 'little_lemon_db',
    'user': 'root',
    'password': ''
}

def create_pool():
    try:
        pool = MySQLConnectionPool(pool_name = "pool_b",
                               pool_size = 2, #default is 5
                               **dbconfig) # unpack argument in dictionary
        print("The connection pool is created with a name: ",pool.pool_name)
        print("The pool size is:",pool.pool_size)
        return pool
    except Error as er:
        print("Error code:", er.errno)
        print("Error message:", er.msg)

open_pool = create_pool()

The connection pool is created with a name:  pool_b
The pool size is: 2


### **Task 2:**

- `Three` guests are trying to book dinner slots simultaneously. 
    - Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.

#### **Task Notes**
* Creating a helper function for insertion and connection closing
    - `Note`: Getting the datetime object and strings was a bit of a pain but got familiar with the dateutil.parser
    - Can likely just pass the string in my format statement but was a good exercise to get familiar with fun insertion errors .. sigh

In [2]:
from dateutil.parser import parse

guest_details = [
    (7, 8, 'Anees', 'Java', '18:00:00', 6),
    (8, 5, 'Bald', 'Vin', '19:00:00', 6),
    (9, 12, 'Jay', 'Kon', '19:30:00', 6)
]

def insert_guest_test(guest):
    # Strings have to be encapsulated when sending
    insert_query_guest_stmt = """
    INSERT INTO Bookings
    VALUES ({}, {}, '{}', '{}', '{}', {});
    """.format(guest[0], guest[1], guest[2], guest[3], parse(guest[4]).strftime('%H:%M:%S'), guest[5])
    print(insert_query_guest_stmt)
    
for guest in guest_details:
    insert_guest_test(guest)
    
def insert_guest(guest):
    insert_query_guest_stmt = """
    INSERT INTO Bookings
    VALUES ({}, {}, '{}', '{}', '{}', {});
    """.format(guest[0], guest[1], guest[2], guest[3], parse(guest[4]).strftime('%H:%M:%S'), guest[5])
    return insert_query_guest_stmt

def close_connections(conn_list):
    for conn in conn_list:
        try:
            conn.close()
        except Error as e:
            print('Error code: ', e.errno)
            print('Error message: ', e.msg)


    INSERT INTO Bookings
    VALUES (7, 8, 'Anees', 'Java', '18:00:00', 6);
    

    INSERT INTO Bookings
    VALUES (8, 5, 'Bald', 'Vin', '19:00:00', 6);
    

    INSERT INTO Bookings
    VALUES (9, 12, 'Jay', 'Kon', '19:30:00', 6);
    


In [3]:
# Now need use try_except handling for insertion based on if new connection available
# active connections (will close at end of cell)
connections = []
for idx, tup in enumerate(guest_details):
    try:
        db_pool_user = f'user_connection_{idx + 1}'
        db_pool_user_connection = open_pool.get_connection()
        print(f'Connection open for : {db_pool_user}')
        # append open connection for subsequent closing following guest table insertion
        connections.append(db_pool_user_connection)
        # create cursor
        db_pool_user_cursor = db_pool_user_connection.cursor()
        # execute query after formatting string with helper function
        db_pool_user_cursor.execute(insert_guest(tup))
        # commit to database
        db_pool_user_connection.commit()
    except Error as e:
        print(e, e.msg) # capture error message
        print(f'For new user attempted connection : {idx + 1}')
        # above is our prints outs for the error message and which new user attempt it failed on, add connection
        open_pool.add_connection()
        db_newc_usr_conn = open_pool.get_connection()
        print(f'New added connection for : user_connection_{idx + 1}')
        # append new open connection
        connections.append(db_newc_usr_conn)
        # create cursor
        db_newc_usr_cursor = db_newc_usr_conn.cursor()
        # execute query w/helper function
        db_newc_usr_cursor.execute(insert_guest(tup))
        # commit to database
        db_newc_usr_conn.commit()

Connection open for : user_connection_1
Connection open for : user_connection_2
Failed getting connection; pool exhausted Failed getting connection; pool exhausted
For new user attempted connection : 3
New added connection for : user_connection_3


In [5]:
# Close Connections
close_connections(connections)

Error code:  -1
Error message:  Failed adding connection; queue is full


### End of Task 2
- The closing of the connections seems to be a bit odd with the mysqlconnector-pool
- Function from Class in Github 
```python
    def close(self):
        """Do not close, but add connection back to pool

        The close() method does not close the connection with the
        MySQL server. The connection is added back to the pool so it
        can be reused.
        """
        self._cnx_pool.add_connection(self._cnx)
        self._cnx = None
```
* Insertions were made however and although ... likely unnecessary the format statement for the insertion was able to use another item of likely future db usage (date util parser for help in formatting accepted values for a date column in a table

### **Task 3**
Create a report containing the following information:
- The `name` and `EmployeeI`D of the Little Lemon manager.
- The `name` and `role` of the employee who receives the highest salary.
- The `number of guests` booked between 18:00 and 20:00.
- The `full name` and `BookingID` of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot

#### `Report Details`
* Each report is in itself an individual query that will be put in subsequent cells for the report query and return from the cursor

In [9]:
# Task 3.1 - Name and EmployeeID of the Little Lemon Manager
# First we need a connection to the db however
try:
    report_gen_conn = open_pool.get_connection()
    report_gen_conn
except Error as e:
    print(e)

In [12]:
# create cursor
report_gen_conn_cursor = report_gen_conn.cursor()
# generate query
name_empid_manager_stmt = """
SELECT
    Name, EmployeeID
FROM Employees
WHERE Role = 'Manager';
"""
# execute query
report_gen_conn_cursor.execute(name_empid_manager_stmt)
# print columns and report
print('Here is our Report for the First Rquest in Task 3')
print(report_gen_conn_cursor.column_names)
for row in report_gen_conn_cursor.fetchall():
    print(row)

Here is our Report for the First Rquest in Task 3
('Name', 'EmployeeID')
('Mario Gollini', 1)


In [13]:
# Task 3.2 - Name & Role of employee who receives the highest salary
name_role_max_sal_stmt = """
SELECT 
    Name, Role
FROM Employees
WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees);
"""
# execute query
report_gen_conn_cursor.execute(name_role_max_sal_stmt)
# print columns and report
print('Here is our Report for the Second Request in Task 3')
print(report_gen_conn_cursor.column_names)
for row in report_gen_conn_cursor.fetchall():
    print(row)

Here is our Report for the Second Request in Task 3
('Name', 'Role')
('Mario Gollini', 'Manager')


In [15]:
# Task 3.3 - Number of Guests booked between 18:00 and 20:00
num_guests_window_stmt = """
SELECT
    COUNT(*) AS num_guests
FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00'
"""
# execute query
report_gen_conn_cursor.execute(num_guests_window_stmt)
# print columns and report
print('Here is our Report for the Third Request in Task 3')
print(report_gen_conn_cursor.column_names[0]) # just need the first one 
for row in report_gen_conn_cursor.fetchall():
    print(row[0]) # only one value so can index first value

Here is our Report for the Third Request in Task 3
num_guests
7


In [16]:
# Task 3.4 - Full Name & BookingiD of guest waiting to be seated
# EmployeeID == 6 'Receptionist', will sort in chronological order 
guest_to_be_seated_chron_stmt = """
SELECT
    CONCAT(GuestFirstName, ' ', GuestLastName) AS guest_full_name,
    BookingID
FROM Bookings
WHERE EmployeeID = 6
ORDER BY BookingSlot;
"""
# execute query
report_gen_conn_cursor.execute(guest_to_be_seated_chron_stmt)
# print columns and report
print('Here is our Report for the Fourth Request in Task 3')
print(report_gen_conn_cursor.column_names)
for row in report_gen_conn_cursor.fetchall():
    print(row) 

Here is our Report for the Fourth Request in Task 3
('guest_full_name', 'BookingID')
('Anees Java', 7)
('Bald Vin', 8)
('Jay Kon', 9)


### **Task 4**
Create a stored procedure named BasicSalesReport that returns the following statistics: 
- Total sales
- Average sale
- Minimum bill paid
- Maximum bill paid

In [17]:
sp_bsr_stmt = """
CREATE PROCEDURE BasicSalesReport()
BEGIN
    SELECT
        COUNT(*) AS total_sales,
        ROUND(AVG(BillAmount),2) AS average_sale,
        MIN(BillAmount) AS min_bill_paid,
        MAX(BillAmount) AS max_bill_paid
    FROM Orders;
END
"""
# run query to create procuedure
report_gen_conn_cursor.execute(sp_bsr_stmt)
# Call procedure
report_gen_conn_cursor.callproc('BasicSalesReport')
# stores results in a variable called dataset
dataset = next(report_gen_conn_cursor.stored_results())
# column_names
print(dataset.column_names)
# Print sorted data with for loop
for row in dataset.fetchall():
    print(row)

('total_sales', 'average_sale', 'min_bill_paid', 'max_bill_paid')
(5, Decimal('48.60'), 37, 86)
